# Recolha e análise de sequência de referência.

A sequência foi recolhida na base de dados *Nucleotide* e foi guardada num ficheiro de formato GeneBank. Porém nesta base de dados não foi possível encontrar um sequência referente ao gene de interesse, sendo como tal necessário recolher desta a partir da sequência inteira do vírus, sendo a sequência escolhida a "MN908947.3".

In [5]:
import os
from Bio import SeqIO, Entrez

a = %pwd
wd = (a.rsplit('/',2))[0]

seq_id = "MN908947.3"
gene = "ORF7a"
protein = "P0DTC7"

In [6]:
Entrez.email = "pedroalex123@gmail.com"
handle = Entrez.efetch(db="nucleotide",
                       id=seq_id,
                       rettype="gb",
                       retmode="text")

file_path = os.path.join(wd,"data/reference/", gene, f"{seq_id}.gb")
with open(file_path, 'w') as file:
    file.write(handle.read())

In [7]:
path = os.path.join(wd, "data/reference",gene,f"{seq_id}.gb")
seq_r = SeqIO.read(path, format="genbank")
for Key, Value in seq_r.annotations.items():
    print(Key,": ", Value)


molecule_type :  ss-RNA
topology :  linear
data_file_division :  VRL
date :  18-MAR-2020
accessions :  ['MN908947']
sequence_version :  3
keywords :  ['']
source :  Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2)
organism :  Severe acute respiratory syndrome coronavirus 2
taxonomy :  ['Viruses', 'Riboviria', 'Orthornavirae', 'Pisuviricota', 'Pisoniviricetes', 'Nidovirales', 'Cornidovirineae', 'Coronaviridae', 'Orthocoronavirinae', 'Betacoronavirus', 'Sarbecovirus']
references :  [Reference(title='A new coronavirus associated with human respiratory disease in China', ...), Reference(title='Direct Submission', ...)]
comment :  On Jan 17, 2020 this sequence version replaced MN908947.2.
structured_comment :  OrderedDict([('Assembly-Data', OrderedDict([('Assembly Method', 'Megahit v. V1.1.3'), ('Sequencing Technology', 'Illumina')]))])


## Extração de conhecimento do GenBank

O ficheiro .gb contém várias, entre as quais podemos confirmar o organismo, a taxonomia do mesmo e referências bibliográficas associadas a esta sequência. Porém a existência de *features* vai permitir conhecer melhor a sequência e determinar a localização do nosso gene de interesse.

### Features

In [8]:
interest_features = []

for feature in seq_r.features:
    if feature.type in ["gene", "CDS"]:
        if feature.qualifiers["gene"][0] == gene:
            interest_features.append(feature)
            print(feature.type)
            print(feature.location)
            print(feature.ref_db)
            for Key,Value in feature.qualifiers.items():
                print(Key,": ", Value)
            print(" ")

gene
[27393:27759](+)
None
gene :  ['ORF7a']
 
CDS
[27393:27759](+)
None
gene :  ['ORF7a']
codon_start :  ['1']
product :  ['ORF7a protein']
protein_id :  ['QHD43421.1']
translation :  ['MKIILFLALITLATCELYHYQECVRGTTVLLKEPCSSGTYEGNSPFHPLADNKFALTCFSTQFAFACPDGVKHVYQLRARSVSPKLFIRQEEVQELYSPIFLIVAAIVFITLCFTLKRKTE']
 


Realizando a filtração destes features foi possível encontrar tanto uma anotação para o gene como a CDS que este representa, ambas indicam uma localização coincidente, que poderá ser usada para *splicing* deste gene.


### Splicing do gene de interesse.

In [10]:
from Bio.SeqRecord import SeqRecord


gene_seq = seq_r.seq[interest_features[0].location.nofuzzy_start:interest_features[0].location.nofuzzy_end]
gene_OBJ = SeqRecord(gene_seq, f"{seq_id} | Reference sequence {gene} gene", "", "")
SeqIO.write(gene_OBJ, os.path.join(wd,"data/reference", gene, f"{gene}_nu.fasta"), "fasta")

print(gene_seq)

ATGAAAATTATTCTTTTCTTGGCACTGATAACACTCGCTACTTGTGAGCTTTATCACTACCAAGAGTGTGTTAGAGGTACAACAGTACTTTTAAAAGAACCTTGCTCTTCTGGAACATACGAGGGCAATTCACCATTTCATCCTCTAGCTGATAACAAATTTGCACTGACTTGCTTTAGCACTCAATTTGCTTTTGCTTGTCCTGACGGCGTAAAACACGTCTATCAGTTACGTGCCAGATCAGTTTCACCTAAACTGTTCATCAGACAAGAGGAAGTTCAAGAACTTTACTCTCCAATTTTTCTTATTGTTGCGGCAATAGTGTTTATAACACTTTGCTTCACACTCAAAAGAAAGACAGAATGA


### Extração de informação relativa à proteina

Podemos ficar a conhecer melhor a proteína, seja a identificação da ORF que permite obter a sua tradução, como a sequência desta e mais importante um ID, que poderá ser usado para pesquisar noutras bases de dados como o Uniprot.

In [11]:
from Bio.Seq import Seq

prot_seq = Seq(interest_features[1].qualifiers['translation'][0])
prot_OBJ = SeqRecord(prot_seq, interest_features[1].qualifiers['protein_id'][0] ,interest_features[1].qualifiers['product'][0], interest_features[1].qualifiers['product'][0])
SeqIO.write(prot_OBJ, os.path.join(wd,"data/reference", gene, f"{gene}_prot.fasta"), "fasta")

print(prot_seq)

MKIILFLALITLATCELYHYQECVRGTTVLLKEPCSSGTYEGNSPFHPLADNKFALTCFSTQFAFACPDGVKHVYQLRARSVSPKLFIRQEEVQELYSPIFLIVAAIVFITLCFTLKRKTE


## Extração de conhecimento do Uniprot

### Anotações

Como referido anteriormente, com base no identificador para a proteína, foi possível encontrar na *Uniprot* o id nesse base de dados a qual esta proteína corresponde, nomeadamente P0DTC7.

Descarregando o ficheiro XML e abrindo com o SeqIO podemos então analisar a função desta proteína e algumas das suas anotações e features.

In [13]:
from wget import download

url = "https://www.uniprot.org/uniprot/{0}.xml".format(protein)
path = os.path.join(wd, "data/reference", gene,f"{protein}.xml")

download(url, path)
prot_ref = SeqIO.read(path, format="uniprot-xml")

Alguns dos elementos mais importantes nestas bases de dados são os comentários, estes incluem informações como função, interações, localização subcelular e domínios que estejam presente:

In [14]:
import re

for annotation in prot_ref.annotations.keys():
    if re.match("(comment_)[a-z]*", annotation):
        print(annotation.split("_",1)[1],":")
        print(prot_ref.annotations[annotation][0], "\n")

function :
Plays a role as antagonist of host tetherin (BST2), disrupting its antiviral effect. Acts by binding to BST2 thereby interfering with its glycosylation. May suppress small interfering RNA (siRNA). May bind to host ITGAL, thereby playing a role in attachment or modulation of leukocytes. 

subunit :
Interacts with the spike glycoprotein. Interacts with M protein. Interacts with E protein. Interacts with the ORF3a protein. Interacts with human SGT. Interacts with host ITGAL. Interacts with host BST2. 

subcellularlocation_location :
Virion 

subcellularlocation_topology :
Single-pass membrane protein 

domain :
The di-lysine motif confers endoplasmic reticulum localization for type I membrane proteins. 



### Features

Muitas vezes estas informações alertam-nos para a presença de features no sequência, como é o caso da informação relativa ao domínio, encontrado no passo anterior.
Estas features têm diferentes níveis de evidência, que podem ser filtrados, neste caso nós escolhemos o nível 2.

Nota: Todas as localizações têm um início adiantado uma posição, devido ao processamento de dados usando o python.

In [15]:
for feature in prot_ref.features:
    if int(feature.qualifiers["evidence"]) >= 2:
        print("feature: ", feature.type)
        print("location: ", feature.location)
        print("seq: ", prot_ref.seq[feature.location.nofuzzy_start:feature.location.nofuzzy_end])
        for qual in feature.qualifiers.keys():
            if qual != "type":
                print(qual,": ", feature.qualifiers[qual])
        print("\n")

feature:  signal peptide
location:  [0:15]
seq:  MKIILFLALITLATC
evidence :  3


feature:  chain
location:  [15:121]
seq:  ELYHYQECVRGTTVLLKEPCSSGTYEGNSPFHPLADNKFALTCFSTQFAFACPDGVKHVYQLRARSVSPKLFIRQEEVQELYSPIFLIVAAIVFITLCFTLKRKTE
description :  ORF7a protein
id :  PRO_0000449654
evidence :  3


feature:  transmembrane region
location:  [95:116]
seq:  LYSPIFLIVAAIVFITLCFTL
description :  Helical
evidence :  3


feature:  domain
location:  [15:81]
seq:  ELYHYQECVRGTTVLLKEPCSSGTYEGNSPFHPLADNKFALTCFSTQFAFACPDGVKHVYQLRARS
description :  X4e
evidence :  4


feature:  short sequence motif
location:  [116:121]
seq:  KRKTE
description :  Di-lysine motif
evidence :  2


feature:  disulfide bond
location:  [22:58]
seq:  CVRGTTVLLKEPCSSGTYEGNSPFHPLADNKFALTC
evidence :  4


feature:  disulfide bond
location:  [34:67]
seq:  CSSGTYEGNSPFHPLADNKFALTCFSTQFAFAC
evidence :  4




Tanto as anotações como as *features*, identificadas, nestes ficheiros. Aquelas relativamente aos dominios e *motifs* podem ser úteis para uma melhor compreensão das anotações dos dados a ser analisados, e em especifico a interação entre os diferentes genes.